In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth) 

    100% |████████████████████████████████| 993kB 20.4MB/s 


In [ ]:
%cd "/content/drive/My Drive/fine_tune_myself"

/content/drive/My Drive/fine_tune_myself


In [ ]:
from pickle import load
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint
 
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text
 
# load a pre-defined list of photo identifiers
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split('.')[0]
		dataset.append(identifier)
	return set(dataset)
 
# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			# store
			descriptions[image_id].append(desc)
	return descriptions
 
# load photo features
def load_photo_features(filename, dataset):
	# load all features
	all_features = load(open(filename, 'rb'))
	# filter features
	features = {k: all_features[k] for k in dataset}
	return features
 
# covert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc
 
# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
	lines = to_lines(descriptions)
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer
 
# calculate the length of the description with the most words
def max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)
 
# create sequences of images, input sequences and output words for an image
def create_sequences(tokenizer, max_length, desc_list, photo):
	X1, X2, y = list(), list(), list()
	# walk through each description for the image
	for desc in desc_list:
		# encode the sequence
		seq = tokenizer.texts_to_sequences([desc])[0]
		# split one sequence into multiple X,y pairs
		for i in range(1, len(seq)):
			# split into input and output pair
			in_seq, out_seq = seq[:i], seq[i]
			# pad input sequence
			in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
			# encode output sequence
			out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
			# store
			X1.append(photo)
			X2.append(in_seq)
			y.append(out_seq)
	return np.array(X1), np.array(X2), np.array(y)
 
# define the captioning model
def define_model(vocab_size, max_length):
  # feature extractor model
  inputs1 = Input(shape=(4096,))
  fe1 = Dropout(0.5)(inputs1)
  fe2 = Dense(256, activation='relu')(fe1)
  # sequence model
  inputs2 = Input(shape=(max_length,))
  se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
  se2 = Dropout(0.5)(se1)
  se3 = LSTM(256)(se2)
  # decoder model
  decoder1 = add([fe2, se3])
  decoder2 = Dense(512, activation='relu')(decoder1)
  decoder3 = Dense(256, activation='relu')(decoder2)
  outputs = Dense(vocab_size, activation='softmax')(decoder3)
	# tie it together [image, seq] [word]
  model = Model(inputs=[inputs1, inputs2], outputs=outputs)
  # compile model
  model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
  # summarize model
  model.summary()
  plot_model(model, to_file='model.png', show_shapes=True)
  return model

def data_generator(descriptions, photos, tokenizer, max_length):
  # loop for ever over images
  while 1:
    for key, desc_list in descriptions.items():
      # retrieve the photo feature
      photo = photos[key]
      photo=photo.reshape(4096)
      in_img, in_seq, out_word = create_sequences(tokenizer, max_length, desc_list, photo)
      yield [[in_img, in_seq], out_word]

Using TensorFlow backend.


In [ ]:
from pickle import load
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers.merge import add
from keras.callbacks import ModelCheckpoint

filename = '/content/drive/My Drive/image cap/Flickr_8k.trainImages.txt'
train = load_set(filename)
test=load_set('/content/drive/My Drive/fine_tune_myself/Flickr_8k.testImages.txt')
print('Dataset: %d' % len(train))
# descriptions
train_descriptions = load_clean_descriptions('/content/drive/My Drive/image cap/descriptions2.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))
# photo features
train_features = load_photo_features('/content/drive/My Drive/fine_tune_myself/fvgg4096best.pkl', train)
# prepare tokenizer
test_desc=load_clean_descriptions('/content/drive/My Drive/image cap/descriptions2.txt', test)
test_features = load_photo_features('/content/drive/My Drive/fine_tune_myself/fvgg4096best.pkl', test)
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# determine the maximum sequence length
max_len = max_length(train_descriptions)
print('Description Length: %d' % max_len)
 
# define the model
model = define_model(vocab_size, max_len)

In [ ]:
epochs = 30
steps = len(train_descriptions)
for i in range(epochs):
  # create the data generator
  generator = data_generator(train_descriptions, train_features, tokenizer, max_len)
  test_gen=data_generator(test_desc, test_features, tokenizer, max_len)
  # fit for one epoch
  model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1,validation_data=test_gen,validation_steps=1000)
  # save model
  model.save('ftv4096_256_merge' + str(i) + '.h5')

Instructions for updating:
Use tf.cast instead.
Epoch 1/1
6000/6000 [==============================] - 734s 122ms/step - loss: 4.3685 - acc: 0.2309 - val_loss: 3.9157 - val_acc: 0.2690
Epoch 1/1
6000/6000 [==============================] - 733s 122ms/step - loss: 3.6235 - acc: 0.2886 - val_loss: 3.7054 - val_acc: 0.2941
Epoch 1/1
6000/6000 [==============================] - 734s 122ms/step - loss: 3.3881 - acc: 0.3075 - val_loss: 3.6400 - val_acc: 0.3026
Epoch 1/1
6000/6000 [==============================] - 727s 121ms/step - loss: 3.2610 - acc: 0.3177 - val_loss: 3.6139 - val_acc: 0.3078
Epoch 1/1
6000/6000 [==============================] - 719s 120ms/step - loss: 3.1770 - acc: 0.3242 - val_loss: 3.6045 - val_acc: 0.3138
Epoch 1/1
6000/6000 [==============================] - 730s 122ms/step - loss: 3.1203 - acc: 0.3298 - val_loss: 3.6125 - val_acc: 0.3122
Epoch 1/1
6000/6000 [==============================] - 737s 123ms/step - loss: 3.0777 - acc: 0.3336 - val_loss: 3.6181 - val_acc: 